Sure! Here's the step-by-step documentation of the strategy:

## RSI Sell Strategy Documentation

### Step 1: Fetch Historical Data

The first step of the strategy involves fetching historical OHLCV (Open, High, Low, Close, Volume) data for the specified trading symbol. The `get_historical_data` function is used for this purpose. It connects to the MetaTrader 5 platform, retrieves historical data with a given timeframe and number of data points, and returns a pandas DataFrame containing the historical data.

### Step 2: Calculate RSI

After fetching historical data, the next step is to calculate the Relative Strength Index (RSI) indicator for the 'close' price column. The `calculate_rsi` function is used to compute the RSI values based on the specified RSI period (default is 14). The RSI is an oscillator that helps identify potential overbought or oversold market conditions.

### Step 3: Find Filling Mode

Before executing any trades, the appropriate filling mode needs to be determined. The `find_filling_mode` function checks for two possible filling modes: FILLING_FOK (Fill or Kill) and FILLING_IOC (Immediate or Cancel). These modes specify how the order should be filled in the market.

### Step 4: Execute Sell Trade

The core of the strategy lies in the `execute_sell_trade` function. This function implements the sell part of the RSI strategy based on specific conditions. The sell trade is executed when the following conditions are met:
- RSI is above 70, indicating an overbought market.
- The candle two periods ago (confirmation candle) is green (close > open).
- The previous candle (immediate previous to the current candle) is red (close < open).
- The current candle (most recent) is also red (close < open).

Once all the sell conditions are satisfied, the function opens a sell trade at the open price of the next red candle. The trade is executed using the MetaTrader 5 API, and the filling mode is determined based on the previously calculated value. The trade remains open until the close of the second red candle after the confirmation candle, and the exit time and exit price are recorded based on the close price of the second red candle.

### Step 5: Run Strategy on Historical Data

Finally, the `run_strategy` function orchestrates the overall strategy execution. It runs the RSI sell strategy on historical data for the specified trading symbol and timeframe. It loops indefinitely, fetching historical data, calculating RSI, and executing sell trades based on the strategy conditions.

### Note and Disclaimer

It's essential to understand that this strategy is provided for illustrative purposes only and should not be considered as financial advice. Trading strategies involve inherent risks, and past performance does not guarantee future results. For real-world trading, thorough testing, risk management, and appropriate stop-loss measures are critical. Always exercise caution and conduct due diligence when engaging in live trading activities.

In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import ta.momentum as momentum
from datetime import datetime
import time

def get_historical_data(symbol, timeframe, number_of_data=1000):
    
    if not mt5.initialize():
        print("initialize() failed ☢️")
        mt5.shutdown()
        return None

    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, number_of_data)

    if rates is None:
        print("Failed to retrieve historical data. ☢️")
        mt5.shutdown()
        return None

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df = df.set_index("time")

    mt5.shutdown()

    return df

def calculate_rsi(df, period=14):
    
    try:
        rsi_indicator = momentum.RSIIndicator(df["close"], window=period)
        df["rsi"] = rsi_indicator.rsi()
    except Exception as e:
        print(f"Error calculating RSI: {e}")

def find_filling_mode(symbol):

    for i in range(2):

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC
        }

        result = mt5.order_check(request)

        if result.comment == "Done":
            print("Trade was closed")
            break

    return i

def execute_sell_trade(df, symbol, lot_size=0.2):
    
    current_bar = df.iloc[-1]
    previous_bar = df.iloc[-2]
    confirmation_bar = df.iloc[-3]

    # Check for sell conditions
    if current_bar["rsi"] > 70 and confirmation_bar["close"] > confirmation_bar["open"] \
            and previous_bar["close"] < previous_bar["open"] \
            and current_bar["close"] < current_bar["open"]:

        if not mt5.initialize():
            print("initialize() failed ☢️")
            mt5.shutdown()
            return

        # Execute the trade on the next red candle (Candle 3 in the description)
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": lot_size,
            "type": mt5.ORDER_TYPE_SELL,
            "price": mt5.symbol_info_tick(symbol).bid,
            "deviation": 10,
            "magic": 0,
            "comment": "RSI Sell Strategy",
            "type_filling": find_filling_mode(symbol),
            "type_time": mt5.ORDER_TIME_GTC
        }

        result = mt5.order_send(request)

        mt5.shutdown()

        if result.comment == "Accepted":
            print("Sell executed")
            # Wait for the trade to close (at the close of the second red candle)
            while True:
                new_bar = get_historical_data(symbol, mt5.TIMEFRAME_M1, 1)
                if new_bar is not None:
                    if new_bar.iloc[0]["close"] < new_bar.iloc[0]["open"]:
                        break
                    time.sleep(1)
            print("Trade closed")
        else:
            print("Error executing the trade")


def run_strategy(symbol, timeframe, lot_size=0.2, data_length=1000, period=14):
    
    while True:

        try:
            df = get_historical_data(symbol, timeframe, data_length)

            if df is not None:
                calculate_rsi(df, period)

                execute_sell_trade(df, symbol, lot_size)

        except Exception as e:
            print(f"Error executing the strategy: {e}")

        sleep_duration = 60
        print("Waiting for a new opportunity")
        time.sleep(sleep_duration)

if __name__ == "__main__":

    symbol = "Boom 1000 Index"
    timeframe = mt5.TIMEFRAME_M1

    lot_size = 0.5

    data_length = 1000
    period = 14

    run_strategy(symbol, timeframe, lot_size, data_length, period)
